In [40]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import PorterStemmer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import re, string, random
import math
import random
from collections import defaultdict
from pprint import pprint
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import string


import numpy as np
import pandas as pd

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\utkar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utkar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
reviews = pd.read_csv(r"C:\Users\utkar\Desktop\Research Paper\amazon_reviews_sample.csv")
df = pd.DataFrame(reviews)
df.tail()

,Unnamed: 0,score,review
9995,9995,1,A revelation of life in small town America in...
9996,9996,1,Great biography of a very interesting journal...
9997,9997,0,Interesting Subject; Poor Presentation: You'd...
9998,9998,0,Don't buy: The box looked used and it is obvi...
9999,9999,1,Beautiful Pen and Fast Delivery.: The pen was...


In [50]:
#Lemmatization - To be done before noise removal to preserve the context of the words of the sentence
WNlemmatizer = WordNetLemmatizer()
word_tokens = [word_tokenize(sentence) for sentence in df.review]
print(word_tokens[:100])
#lemmatized_tokens = [[WNlemmatizer.lemmatize(word) for word in sentence] for sentence in word_tokens]

[['Stuning', 'even', 'for', 'the', 'non-gamer', ':', 'This', 'sound', 'track', 'was', 'beautiful', '!', 'It', 'paints', 'the', 'senery', 'in', 'your', 'mind', 'so', 'well', 'I', 'would', 'recomend', 'it', 'even', 'to', 'people', 'who', 'hate', 'vid', '.', 'game', 'music', '!', 'I', 'have', 'played', 'the', 'game', 'Chrono', 'Cross', 'but', 'out', 'of', 'all', 'of', 'the', 'games', 'I', 'have', 'ever', 'played', 'it', 'has', 'the', 'best', 'music', '!', 'It', 'backs', 'away', 'from', 'crude', 'keyboarding', 'and', 'takes', 'a', 'fresher', 'step', 'with', 'grate', 'guitars', 'and', 'soulful', 'orchestras', '.', 'It', 'would', 'impress', 'anyone', 'who', 'cares', 'to', 'listen', '!', '^_^'], ['The', 'best', 'soundtrack', 'ever', 'to', 'anything', '.', ':', 'I', "'m", 'reading', 'a', 'lot', 'of', 'reviews', 'saying', 'that', 'this', 'is', 'the', 'best', "'game", 'soundtrack', "'", 'and', 'I', 'figured', 'that', 'I', "'d", 'write', 'a', 'review', 'to', 'disagree', 'a', 'bit', '.', 'This', '

In [51]:
tagged_tokens = [pos_tag(word) for word in word_tokens]

In [52]:
print(tagged_tokens[:100])

[[('Stuning', 'VBG'), ('even', 'RB'), ('for', 'IN'), ('the', 'DT'), ('non-gamer', 'JJ'), (':', ':'), ('This', 'DT'), ('sound', 'NN'), ('track', 'NN'), ('was', 'VBD'), ('beautiful', 'JJ'), ('!', '.'), ('It', 'PRP'), ('paints', 'VBZ'), ('the', 'DT'), ('senery', 'NN'), ('in', 'IN'), ('your', 'PRP$'), ('mind', 'NN'), ('so', 'RB'), ('well', 'RB'), ('I', 'PRP'), ('would', 'MD'), ('recomend', 'VB'), ('it', 'PRP'), ('even', 'RB'), ('to', 'TO'), ('people', 'NNS'), ('who', 'WP'), ('hate', 'VBP'), ('vid', 'NNS'), ('.', '.'), ('game', 'NN'), ('music', 'NN'), ('!', '.'), ('I', 'PRP'), ('have', 'VBP'), ('played', 'VBN'), ('the', 'DT'), ('game', 'NN'), ('Chrono', 'NNP'), ('Cross', 'NNP'), ('but', 'CC'), ('out', 'IN'), ('of', 'IN'), ('all', 'DT'), ('of', 'IN'), ('the', 'DT'), ('games', 'NNS'), ('I', 'PRP'), ('have', 'VBP'), ('ever', 'RB'), ('played', 'VBN'), ('it', 'PRP'), ('has', 'VBZ'), ('the', 'DT'), ('best', 'JJS'), ('music', 'NN'), ('!', '.'), ('It', 'PRP'), ('backs', 'VBZ'), ('away', 'RB'), ('fr

In [88]:
def lemmatize_sentence(tokens):
    lemmatized_sentence = []
    for word,tag in tokens:
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(WNlemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

In [90]:
lemmatized_tokens = [lemmatize_sentence(tokens) for tokens in tagged_tokens]
print(lemmatized_tokens[:100])

[['Stuning', 'even', 'for', 'the', 'non-gamer', ':', 'This', 'sound', 'track', 'be', 'beautiful', '!', 'It', 'paint', 'the', 'senery', 'in', 'your', 'mind', 'so', 'well', 'I', 'would', 'recomend', 'it', 'even', 'to', 'people', 'who', 'hate', 'vid', '.', 'game', 'music', '!', 'I', 'have', 'play', 'the', 'game', 'Chrono', 'Cross', 'but', 'out', 'of', 'all', 'of', 'the', 'game', 'I', 'have', 'ever', 'play', 'it', 'have', 'the', 'best', 'music', '!', 'It', 'back', 'away', 'from', 'crude', 'keyboarding', 'and', 'take', 'a', 'fresh', 'step', 'with', 'grate', 'guitar', 'and', 'soulful', 'orchestra', '.', 'It', 'would', 'impress', 'anyone', 'who', 'care', 'to', 'listen', '!', '^_^'], ['The', 'best', 'soundtrack', 'ever', 'to', 'anything', '.', ':', 'I', "'m", 'read', 'a', 'lot', 'of', 'review', 'say', 'that', 'this', 'be', 'the', 'best', "'game", 'soundtrack', "'", 'and', 'I', 'figure', 'that', 'I', "'d", 'write', 'a', 'review', 'to', 'disagree', 'a', 'bit', '.', 'This', 'in', 'my', 'opinino',

In [91]:
sentences = [' '.join(r) for r in lemmatized_tokens]
df['Lemmatized'] = sentences
df.head(20)

,Unnamed: 0,score,review,Lemmatized,cleantext
0,0,1,Stuning even for the non-gamer: This sound tr...,Stuning even for the non-gamer : This sound tr...,stuning even non gamer sound track beautiful p...
1,1,1,The best soundtrack ever to anything.: I'm re...,The best soundtrack ever to anything . : I 'm ...,best soundtrack ever anything read lot review ...
2,2,1,Amazing!: This soundtrack is my favorite musi...,Amazing ! : This soundtrack be my favorite mus...,amazing soundtrack favorite music time hand in...
3,3,1,Excellent Soundtrack: I truly like this sound...,Excellent Soundtrack : I truly like this sound...,excellent soundtrack truly like soundtrack enj...
4,4,1,"Remember, Pull Your Jaw Off The Floor After H...","Remember , Pull Your Jaw Off The Floor After H...",remember pull jaw floor hearing play game know...
5,5,1,an absolute masterpiece: I am quite sure any ...,an absolute masterpiece : I be quite sure any ...,absolute masterpiece quite sure actually take ...
6,6,0,"Buyer beware: This is a self-published book, ...","Buyer beware : This be a self-published book ,...",buyer beware self published book want know rea...
7,7,1,Glorious story: I loved Whisper of the wicked...,Glorious story : I love Whisper of the wicked ...,glorious story love whisper wicked saint story...
8,8,1,A FIVE STAR BOOK: I just finished reading Whi...,A FIVE STAR BOOK : I just finish read Whisper ...,five star book finish read whisper wicked sain...
9,9,1,Whispers of the Wicked Saints: This was a eas...,Whispers of the Wicked Saints : This be a easy...,whispers wicked saints easy read book make wan...


In [92]:
stop = set(stopwords.words('english'))

In [93]:
def clean_document(doco):
    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)
    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')
    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])
    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    doco_clean = ([i for i in doco_clean if i not in stop])
    doco_clean = ([i for i in doco_clean if i.isalpha()]) # removing numeric
    #doco_clean = ([[word for word in item if word.isalpha()] for item in doco_clean])
#     doco_clean = [spell(word) for word in doco_clean]
#     p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
#     doco_clean = ([spell(k) for k in doco_clean])
    return doco_clean

In [94]:
review_clean = [clean_document(doc) for doc in df['Lemmatized']]
sentences = [' '.join(r) for r in review_clean]
df['cleantext']=sentences
df.tail(20)

,Unnamed: 0,score,review,Lemmatized,cleantext
9980,9980,1,"Magic, on Blu Ray, starrring Anthony Hopkins ...","Magic , on Blu Ray , starrring Anthony Hopkins...",magic blu ray starrring anthony hopkins ann ma...
9981,9981,1,A ventriloquists nightmare: Magic is a timele...,A ventriloquist nightmare : Magic be a timeles...,ventriloquist nightmare magic timeless suspens...
9982,9982,0,great movie massacred by tape quality: One of...,great movie massacre by tape quality : One of ...,great movie massacre tape quality one time fav...
9983,9983,1,Early Hopkins story still sends chills throug...,Early Hopkins story still send chill through v...,early hopkins story still send chill viewer ma...
9984,9984,0,"The Only Dummy Is The Writer: ""Magic"" poses t...",The Only Dummy Is The Writer : `` Magic '' pos...,dummy writer magic pose question whether ventr...
9985,9985,1,"""He's NO Dummy. . ."": Viewing ""Magic"" is when...",`` He 's NO Dummy . . . `` : Viewing `` Magic ...,dummy viewing magic first become aware anthony...
9986,9986,1,Amazingly suspenseful psychological thriller:...,Amazingly suspenseful psychological thriller :...,amazingly suspenseful psychological thriller s...
9987,9987,1,A truly great horror movie: I saw this film l...,A truly great horror movie : I saw this film l...,truly great horror movie saw film last night r...
9988,9988,1,Frightening movie with superb acting by Sir H...,Frightening movie with superb act by Sir Hopki...,frightening movie superb act sir hopkins recol...
9989,9989,1,classic: i got this for my dad. it is super c...,classic : i get this for my dad . it be super ...,classic get dad super creepy worth watch watch...


In [7]:
train_positive_sentiment = df[df.score == 1]
positive_words = [word_tokenize(sentence) for sentence in train_positive_sentiment['cleantext']]
#print(positive_words)
train_negative_sentiment = df[df.score == 0]
negative_words = [word_tokenize(sentence) for sentence in train_negative_sentiment['cleantext']]
#print(negative_words)
print(positive_words)

[['stuning', 'even', 'non', 'gamer', 'sound', 'track', 'beautiful', 'paints', 'senery', 'mind', 'well', 'would', 'recomend', 'even', 'people', 'hate', 'vid', 'game', 'music', 'played', 'game', 'chrono', 'cross', 'games', 'ever', 'played', 'best', 'music', 'backs', 'away', 'crude', 'keyboarding', 'takes', 'fresher', 'step', 'grate', 'guitars', 'soulful', 'orchestras', 'would', 'impress', 'anyone', 'cares', 'listen'], ['best', 'soundtrack', 'ever', 'anything', 'reading', 'lot', 'reviews', 'saying', 'best', 'game', 'soundtrack', 'figured', 'write', 'review', 'disagree', 'bit', 'opinino', 'yasunori', 'mitsuda', 'ultimate', 'masterpiece', 'music', 'timeless', 'listening', 'years', 'beauty', 'simply', 'refuses', 'fade', 'price', 'tag', 'pretty', 'staggering', 'must', 'say', 'going', 'buy', 'much', 'money', 'one', 'feel', 'would', 'worth', 'every', 'penny'], ['amazing', 'soundtrack', 'favorite', 'music', 'time', 'hands', 'intense', 'sadness', 'prisoners', 'fate', 'means', 'played', 'game', 'h

<strong><H1>BOW Testing</H1><br><br>
1. Unigrams</strong>

In [95]:
vect = CountVectorizer(max_features = 5000)
vect.fit(df.cleantext)
vect_bow = vect.transform(df.cleantext)
vect_df = pd.DataFrame(vect_bow.toarray(), columns = vect.get_feature_names())
vect_df.head()

,abandon,ability,able,abroad,absent,absolute,absolutely,absolutly,absorb,absorbed,...,yoruba,young,youth,youtube,zen,zero,zeta,ziggy,zombie,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
y = df.score
X_bow = vect_df

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#Without lemmatization -> 84.25
#With Lemmatization, BOW-uni, preproc, 5000 features -> 83.55.
#Surprisingly after lemmatization the accuracy decreased.

0.8355


<strong>2.Uni+Bigrams</strong>

In [98]:
vect = CountVectorizer(max_features = 5000, ngram_range = (1,2))
vect.fit(df.cleantext)
vect_bow = vect.transform(df.cleantext)
vect_df = pd.DataFrame(vect_bow.toarray(), columns = vect.get_feature_names())
vect_df.head()

,ability,able,able find,able get,absolute,absolutely,absolutely love,absolutely nothing,absorb,absurd,...,young,young child,young woman,youth,zen,zero,zero star,zeta,zeta jones,zombie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
y = df.score
X_bow = vect_df

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#Without Lemmatization + UNi+Bigrams it was -> 85.15
#With Lemma, UNi+bi, preproc -> 83.5
#THis is also less in lemmatization

0.835


<strong><H1>TFIDF</H1><br>
    1.Unigrams</strong>

In [102]:
vect1 = TfidfVectorizer(max_features = 5000)
vect1.fit(df.cleantext)
vect1_tfidf = vect1.transform(df.cleantext)
vect1_df = pd.DataFrame(vect1_tfidf.toarray(), columns = vect1.get_feature_names())
vect1_df.head()

,abandon,ability,able,abroad,absent,absolute,absolutely,absolutly,absorb,absorbed,...,yoruba,young,youth,youtube,zen,zero,zeta,ziggy,zombie,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.111921,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
y = df.score
X_tfidf = vect1_df

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#Without lemmatization it was -> 85.35%
#WIth lemma, uni+bigrams, preprc -> 84.95%

0.8495


<strong>2.Uni+Bigrams</strong>

In [105]:
vect1 = TfidfVectorizer(max_features = 10000, ngram_range = (1,2))
vect1.fit(df.cleantext)
vect1_tfidf = vect1.transform(df.cleantext)
vect1_df = pd.DataFrame(vect1_tfidf.toarray(), columns = vect1.get_feature_names())
vect1_df.head()

,abandon,ability,able,able find,able get,able play,able read,able see,able use,able watch,...,zen garden,zen xtra,zero,zero star,zeta,zeta jones,ziggy,ziggy stardust,zombie,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
y = df.score
X_tfidf = vect1_df

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size = 0.2, random_state = 123)
log_reg = LogisticRegression(solver = 'lbfgs', max_iter = 1000000)
log_reg.fit(X_train, y_train)
y_predicted = log_reg.predict(X_test)
print(accuracy_score(y_test,y_predicted))
#Without lemma it was -> 86.4%
#With lemma it is -> 85.35%

0.8535


In [108]:
WNlemmatizer.lemmatize('furthering','v')

'further'